In [ ]:
from copy import deepcopy
import math
import sys

import matplotlib.pylab as plt
import numpy as np

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

sys.path.append('../crazyflie-audio/python/')

In [ ]:
from generate_results import *

def plot_spectrum(spectrum, degree=0):
    plt.figure()
    if spectrum.shape[0] == 1:
        plt.plot(angles, np.log10(spectrum[0]))
        plt.ylabel('spectrum [-]')
    else:
        plt.pcolormesh(angles, frequencies, np.log10(spectrum))
        plt.ylabel('frequency [Hz]')
    plt.axvline(degree, color='red')
    plt.xlabel('angle [deg]')

# 1. Geometrical setup

In [ ]:
from mic_array import get_square_array, get_uniform_array

baseline = 0.108  # meters, square side of mic array
gt_distance = 10  # meters, distance of source
gt_angle_deg = 50 # angle of ground truth

#mics_drone = get_square_array(baseline=baseline, delta=0) # 4 x 2
mics_drone = get_uniform_array(2, baseline=baseline) # 4 x 2
mics_drone -= np.mean(mics_drone, axis=0) # center the drone

gt_angle_rad = gt_angle_deg * np.pi / 180.0
source = gt_distance * np.array([np.cos(gt_angle_rad), np.sin(gt_angle_rad)])

plt.figure()
plt.scatter(*mics_drone.T)
plt.scatter(*source)
plt.axis('equal')
pass

In [ ]:
### parameters
n_buffer = 2048
time_index = 2000 # idx where signal is non-zero for all positions, found heuristically
angular_velocity_deg = 20 # deg/sec, velocity of drone
times_list = np.array([0, 1, 1.5], dtype=np.float) # in seconds
frequency_hz = 4134.375 # Hz

# make sure times are not a multiple of period.
period = 1/frequency_hz
times_list[1:] += period/2

signal_noise = 1e-3  # noise added to signals
mics_noise = 0 #1e-3  # noise added to mic positions (rigid)
degree_noise = 0 #5 # noise added to each degree position, in degrees
time_quantization = 10 #6 # number of decimal places to keep
time_noise = 0 #1e-6 # noise added to recording times

frequencies = np.fft.rfftfreq(n_buffer, 1/FS)
if frequency_hz not in frequencies:
    raise ValueError(f'frequency_hz not in available frequency bins: {list(frequencies)}')

# 2. Simulate signals at mics

In [ ]:
from audio_stack.beam_former import rotate_mics

np.random.seed(1)

degrees = times_list * angular_velocity_deg
print('degrees:', degrees)
print('recording times in seconds:', times_list)

assert DURATION > max(times_list)

# create noisy versions
mics_drone_noisy = deepcopy(mics_drone)
times_list_noisy = deepcopy(times_list)
degrees_noisy = deepcopy(degrees)
if mics_noise > 0:
    mics_drone_noisy += np.random.normal(scale=mics_noise, size=mics_drone.shape)
if time_noise > 0:
    times_list_noisy += np.random.normal(scale=time_noise, size=times_list.shape)
if time_quantization > 0:
    times_list_noisy = np.round(times_list_noisy, time_quantization)
if degree_noise > 0:
    degrees_noisy += np.random.normal(scale=degree_noise, size=degrees.shape)

# we think we move to "degrees" position but we actually move to degrees_noisy.
mics_list = [rotate_mics(mics_drone_noisy, orientation_deg=degree) for degree in degrees_noisy]

plt.figure()
counter = 0
for degree, mics in zip(degrees, mics_list):
    for j in range(mics.shape[0]):
        plt.scatter(*mics[j], label=f'real {degree:.0f}, mic{j}', color=f"C{counter}")
        counter += 1
        
counter = 0
mics_clean = [rotate_mics(mics_drone, orientation_deg=degree) for degree in degrees]
for degree, mics in zip(degrees, mics_clean):
    for j in range(mics.shape[0]):
        plt.scatter(*mics[j], label=f'theo {degree:.0f}, mic{j}', color=f"C{counter}", marker="x")
        counter += 1
plt.axis('equal')
plt.legend(bbox_to_anchor=[1, 1], loc='upper left')
pass

# generate signals

In [ ]:
from generate_results import generate_signals_analytical

buffer_list = []
for mics, time in zip(mics_list, times_list):
    fig, ax = plt.subplots()
    signals_received =  generate_signals_analytical(
        source, gt_angle_rad, mics, frequency_hz, time, noise=signal_noise, ax=ax) 
    ax.set_xlim(1300, 1400) 
    ax.legend([f"mic{i}" for i in range(mics.shape[0])])
    ax.set_title(f'signals at time {time:.3f}')
    
    buffers = signals_received[:, time_index:time_index + n_buffer]
    buffer_list.append(buffers)
    fig, axs = plt.subplots(signals_received.shape[0])
    n_plot = 50
    fig.suptitle(f'first {n_plot} samples of received buffers')
    for i, ax in enumerate(axs):
        ax.plot(buffers[i, :n_plot], color=f'C{i}')

In [ ]:
from audio_stack.beam_former import BeamFormer

indices = np.where(frequencies == frequency_hz)[0]
for buffer, mics in zip(buffer_list, mics_list) :
    beam_former = BeamFormer(mic_positions=mics)
    
    buffer_f = (np.fft.rfft(buffer).T)[indices, :]
    
    R = beam_former.get_correlation(buffer_f)
    spectrum = beam_former.get_das_spectrum(R, np.array([frequency_hz]))
    #spectrum = beam_former.get_mvdr_spectrum(R, np.array([frequency_hz]))
    
    plt.figure()
    plt.matshow(np.angle(R[0]))
    plt.colorbar()
    
    #plt.figure()
    #plt.plot(BeamFormer.theta_scan_deg, spectrum[0, :])
    
    plt.figure()
    plt.polar(BeamFormer.theta_scan, spectrum[0, :])

# 3. Create "real" multi-mic array

In [ ]:
mics_array = np.concatenate([*mics_list])
signals_multimic = generate_signals_analytical(source, gt_angle_rad, mics_array, frequency_hz, time=0, noise=signal_noise)
#signals_multimic = generate_signals_pyroom(source, source_signal, mics_array.T, time=0, noise=signal_noise)
buffer_multimic = signals_multimic[:, time_index:time_index + n_buffer]

times = np.arange(n_plot) / FS

fig, axs = plt.subplots(buffer_multimic.shape[0], 2, sharex=True)
fig.set_size_inches(10, 10)
for i in range(buffer_multimic.shape[0]): # n_mics
    label=f"mic{i}"
    axs[i, 0].plot(times, buffer_multimic[i, :n_plot], label=label, color=f"C{i}")
    axs[i, 0].set_title(label)
    
buffer_f_multimic = np.fft.rfft(buffer_multimic)[:, indices]
beam_former = BeamFormer(mics_array)
R_multimic = beam_former.get_correlation(buffer_f_multimic.T)
spectrum_multimic = beam_former.get_das_spectrum(R_multimic, [frequency_hz])

plt.figure()
plt.polar(BeamFormer.theta_scan, spectrum_multimic[0, :])

plt.figure()
plt.matshow(np.angle(R_multimic[0]))
    
counter = 0
period = 1.0/frequency_hz
delta_t =  times_list[1] - times_list[0]
print(f'period of signal {period:.2e}')
print('time shift:', delta_t)
print(f'delta_t % period: {delta_t % period :.2e}')

buffer_f_list = []
buffer_f_delayed = np.zeros(buffer_f_multimic.shape, dtype=np.complex)
for j, (degree, buffer, time) in enumerate(zip(degrees, buffer_list, times_list_noisy)):
    
    buffer_f = np.empty((buffer.shape[0], 1), dtype=np.complex)
    
    for i in range(buffer.shape[0]): # n_mics
        label=f"mic{counter}"
        phase_shift_deg = from_0_to_2pi(-2 * np.pi * time * frequency_hz) * 180 / np.pi
        time_shift = period * phase_shift_deg / 360
        
        axs[counter, 1].axvline(time_shift)
        axs[counter, 1].plot(times, buffer[i, :n_plot], color=f"C{counter}")
        axs[counter, 1].set_title(label)
        
        buffer_f[i, :] = np.fft.rfft(buffer[i])[indices]
        exp_delay = np.exp(-2j * np.pi * time * frequency_hz)
        buffer_f_delayed[counter] = exp_delay * buffer_f[i, :]
        counter += 1
        
    buffer_f_list.append(buffer_f)
        
beam_former = BeamFormer(mics_array)
R_delayed = beam_former.get_correlation(buffer_f_delayed.T)
spectrum_delayed = beam_former.get_das_spectrum(R_delayed, [frequency_hz])

plt.figure()
plt.polar(BeamFormer.theta_scan, spectrum_delayed[0, :])

plt.figure()
plt.matshow(np.angle(R_delayed[0]))

# 4. DOA estimation

In [ ]:
# signals with delay correction

frequencies = np.array([frequency_hz])

beam_former = BeamFormer(mic_positions=mics_drone)
beam_former.init_multi_estimate(frequencies, combination_n=len(degrees))
beam_former.init_dynamic_estimate(frequencies, combination_n=len(degrees), 
                                  combination_method='product', normalization_method='none')

signals_f_list_delayed = []

plt.figure()
for i, (sig_f, time) in enumerate(zip(buffer_f_list, times_list_noisy)):
    beam_former.add_to_multi_estimate(sig_f.T, frequencies, time, degrees[i])
    
    spec = beam_former.add_signals_to_dynamic_estimates(sig_f.T, frequencies, degrees[i])
    
    plt.polar(BeamFormer.theta_scan, spec[0, :])
    
plt.title('shifted spectra')

spectrum_multi = beam_former.get_multi_estimate(method='das')
spectrum_dynamic = beam_former.get_dynamic_estimate()

plt.figure()
plt.polar(BeamFormer.theta_scan, spectrum_multi[0, :])
plt.title('spectrum_multi')

plt.figure()
plt.polar(BeamFormer.theta_scan, spectrum_dynamic[0, :])
plt.title('spectrum_dynamic')

# Full pipeline

In [ ]:
import pandas as pd

#fname = 'first_test.pkl' # linear array
fname = 'square_test.pkl' # square array
df = pd.read_pickle(fname)
freq_i = 0

label_list = ['spectrum_combined', 'spectrum_raw', 'spectrum_multimic']

n_degree_noise = len(df.degree_noise.unique())
n_it = len(df.it.unique())
boxplot_mat = np.empty((len(label_list), n_it, n_degree_noise)) # each slice will be one boxplot

for d, (degree_noise, df_degree_noise) in enumerate(df.groupby('degree_noise')):
    fig, axs = plt.subplots(1, len(label_list), sharey=True, sharex=True)
    fig.set_size_inches(10, 3)
    for counter, (__, row) in enumerate(df_degree_noise.iterrows()):
        for l, label in enumerate(label_list):
            if counter <= 3:
                axs[l].semilogy(angles, row[label][freq_i])
                
            unwrap_angles = np.array([angles[np.argmax(row[label][freq_i])], gt_angle_deg]) / 180 * np.pi
            unwrap_angles = np.unwrap(unwrap_angles)
            error = abs(unwrap_angles[1] - unwrap_angles[0])*180/np.pi
            
            boxplot_mat[l, counter, d] = error
        
    for label, ax in zip(label_list, axs):
        ax.axvline(gt_angle_deg, color='black', ls=':')
        ax.set_title(f'degree noise {degree_noise}')
        ax.set_xlabel('angle [deg]')
        ax.set_ylabel('spectrum [-]')
        ax.set_title(label)

In [ ]:
fig_box, ax_box = plt.subplots(1, len(label_list), sharex=True, sharey=True)
fig_box.set_size_inches(10, 5)
for l, label in enumerate(label_list):
    ax_box[l].boxplot(boxplot_mat[l], positions=df.degree_noise.unique(), widths=1.0)
    ax_box[l].set_title(label)